In [50]:
from gmpy2.gmpy2 import random_state

# Quick Start: Causal Analysis
'''
This notebook demonstrates:
1. Load & clean data
2. Feature selection (IAMB)
3. Causal discovery (PC)
4. Effect estimation (ATE)
5. Visualization (DAG)
'''

'\nThis notebook demonstrates:\n1. Load & clean data\n2. Feature selection (IAMB)\n3. Causal discovery (PC)\n4. Effect estimation (ATE)\n5. Visualization (DAG)\n'

In [51]:
import sys, os

# 1. Compute project root: one level up from the notebook folder
proj_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# 2. Prepend it to sys.path
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)

In [52]:
import pandas as pd
from causallearn.utils.cit import kci
from causal import preprocess
from causal import restriction
from causal import causal_discovery as cd
from causal import visualization as vis
from causal import utils
from causallearn.search.FCMBased.lingam.utils import make_dot
from causallearn.search.FCMBased import lingam
from dowhy import CausalModel
from causallearn.utils.GraphUtils import GraphUtils

In [53]:
if __name__ == '__main__':

    '''
    1. Data Loading and Preprocessing
    '''

    path = '../Dataset/veremi_extension_simple.csv'
    data_origin = pd.read_csv(path)
    data_origin = data_origin.sample(n=45000, random_state=42)

    # filter Ddos and normal data
    # data_origin = data_origin[data_origin['class'].isin([0, 11, 12, 13, 16, 17])]

    # filter fake data attack and normal data
    # data_origin = data_origin[data_origin['class'].isin([0, 1, 2, 3, 4, 5, 6, 7, 8])]

    # filter sybil attack and normal data
    # data_origin = data_origin[data_origin['class'].isin([0, 14, 15, 16, 17])]
    print(data_origin.head(5))
    print('*-' * 50)


        type     sendTime  sender  senderPseudo  messageID  class  \
781974     4  64786.79042  102705     101027056  314608216      0   
937737     4  41423.20153   53703      10537034  162475454      0   
907828     4  40404.51425   52437      10524374  159087544      0   
784628     4  64742.75607  102843     101028436  314056141      0   
662460     4  63359.25486   97329      10973296  291942663      0   

               posx        posy  posz       spdx      spdy  spdz      aclx  \
781974   766.006969  388.626455     0 -11.730578 -4.089087     0 -0.020099   
937737   937.440493  885.596937     0   7.065199  6.790865     0  0.046714   
907828   212.555763  393.407409     0  -1.114930  6.797425     0 -0.272466   
784628  1265.321975  975.176831     0 -11.239370  0.950841     0  0.235476   
662460   202.301959  558.407352     0   0.000346  0.000346     0  0.000602   

            acly  aclz      hedx      hedy  hedz  Attack      Attack_type  
781974 -0.005977     0 -0.999999  0.0011

In [54]:
# Data Cleaning:
drop_column = ['type','Attack','Attack_type']
data_processed = preprocess.clean(data_origin, drop_column=drop_column, drop_na=True, data_numerical=True)

# Standardize features, target keep same as original data_processed:
data_processed = preprocess.standardize(data_processed, ['class','sendTime','sender','senderPseudo','messageID'])

# Combine axis related data such as pos, spd etc. by using M = \sqrt{X^2 + Y^2 + Z^2}
data_processed = preprocess.add_vector_magnitude_column(data_processed, ['posx', 'posy', 'posz'], 'pos')
data_processed = preprocess.add_vector_magnitude_column(data_processed, ['spdx', 'spdy', 'spdz'], 'spd')
data_processed = preprocess.add_vector_magnitude_column(data_processed, ['aclx', 'acly', 'aclz'], 'acl')
data_processed = preprocess.add_vector_magnitude_column(data_processed, ['hedx', 'hedy', 'hedz'], 'hed')
data_processed.drop(
    columns=['posx', 'posy', 'posz', 'spdx', 'spdy', 'spdz', 'aclx', 'acly', 'aclz', 'hedx', 'hedy', 'hedz','sender'],
    inplace=True
)

# ID mapping to 0-N
data_processed['senderPseudo'] = data_processed['senderPseudo'].astype('category').cat.codes
data_processed['messageID'] = data_processed['messageID'].astype('category').cat.codes

# …run two separate CausalModel objects with *_z columns as treatment

with pd.option_context('display.max_columns', None):
    print(data_processed)
    print(type(data_processed))
print('*-' * 50)

           sendTime  senderPseudo  messageID  class       pos       spd  \
781974  64786.79042          6706      38496      0  1.022918  1.473916   
937737  41423.20153          2497      16120      0  1.098519  0.651639   
907828  40404.51425          2309      14638      0  1.351199  0.513902   
784628  64742.75607          6726      38369      0  1.952415  1.286059   
662460  63359.25486          5348      33326      0  1.084484  0.308602   
...             ...           ...        ...    ...       ...       ...   
328814  27951.75840          1315       7757      0  1.667238  1.181565   
42373   72684.43262          7048      40414      0  1.752671  0.286874   
931094  41187.16659          2450      15808      0  0.772786  1.250254   
917513  40729.30743          2370      15145      0  1.086405  1.074791   
417651  51045.91705          3037      19215      3  2.139895  1.562077   

             acl       hed  
781974  0.120268  1.383531  
937737  0.040893  1.490228  
907828  1.43

In [55]:
# X = data_processed.iloc[:, 1:].copy()     # 8 features
# y = data_processed.iloc[:, 0].copy()
#
y = data_processed['class'].copy()
X = data_processed.drop(columns='class')

# print(X)
# print('*-' * 50)
# print(y)
# print('*-' * 50)

df = pd.concat([X, y.rename('class')], axis=1)
print(df)
node_names = df.columns.tolist()
print(node_names)

zeros = df.columns[df.var()==0]
print("zero var column：", zeros.tolist())

corr = df.corr().abs()
perfect_pairs = [(i,j) for i in corr.columns for j in corr.columns
                 if i!=j and corr.loc[i,j]==1.0]
print("corr column：", perfect_pairs)

           sendTime  senderPseudo  messageID       pos       spd       acl  \
781974  64786.79042          6706      38496  1.022918  1.473916  0.120268   
937737  41423.20153          2497      16120  1.098519  0.651639  0.040893   
907828  40404.51425          2309      14638  1.351199  0.513902  1.432670   
784628  64742.75607          6726      38369  1.952415  1.286059  0.171871   
662460  63359.25486          5348      33326  1.084484  0.308602  0.100438   
...             ...           ...        ...       ...       ...       ...   
328814  27951.75840          1315       7757  1.667238  1.181565  1.608972   
42373   72684.43262          7048      40414  1.752671  0.286874  2.206176   
931094  41187.16659          2450      15808  0.772786  1.250254  0.365118   
917513  40729.30743          2370      15145  1.086405  1.074791  0.301494   
417651  51045.91705          3037      19215  2.139895  1.562077  0.191178   

             hed  class  
781974  1.383531      0  
937737  1.4

In [56]:
'''
2.  Background knowledge creation
'''
bk_pc = restriction.PC_BGKnowledge(df, X, 'class')
bk_DirectLiNGAM = restriction.DirectLiNGAM_BGKnowledge(node_names, 'class')
# print(bk_DirectLiNGAM)

<class 'causallearn.utils.PCUtils.BackgroundKnowledge.BackgroundKnowledge'>


In [57]:
'''
3.  Algorithm for causal discovery
'''

'''3.1 Constrained Based'''
# PC algorithm with Kernal-based independence test
cg_pc = cd.pc_algorithm(
    df,
    indep_test_func=kci,
    alpha = 0.01,
    uc_rule = 1,
    max_k = 2,
    background_knowledge = bk_pc,
    node_names = node_names
)

# Visualize the PC graph：
# vis.causal_graph(cg_pc, 'PC')
pdy = GraphUtils.to_pydot(cg_pc.G)
print(type(pdy))
pdy.write_png('PC.png')


# FCI algorithm with Kernal-based independence test
cg_fci, edges = cd.fci_algorithm(
    df,
    indep_test_func=kci,
    alpha=0.01,
    depth=-1,
    max_path_length=-1,
    verbose=False,
    show_progress=True,
    background_knowledge = bk_pc,
    node_names = node_names
)
pdy = GraphUtils.to_pydot(cg_fci)
pdy.write_png('FCI.png')

Depth=5, working on node 7: 100%|██████████| 8/8 [00:00<00:00, 2852.06it/s]

<class 'pydot.core.Dot'>



Depth=0, working on node 7: 100%|██████████| 8/8 [00:00<00:00, 1728.99it/s]


Starting BK Orientation.
Orienting edge (Knowledge): senderPseudo --> class
Orienting edge (Knowledge): pos --> class
Orienting edge (Knowledge): spd --> class
Orienting edge (Knowledge): acl --> class
Orienting edge (Knowledge): hed --> class
Finishing BK Orientation.
Starting BK Orientation.
Orienting edge (Knowledge): senderPseudo --> class
Orienting edge (Knowledge): pos --> class
Orienting edge (Knowledge): spd --> class
Orienting edge (Knowledge): acl --> class
Orienting edge (Knowledge): hed --> class
Finishing BK Orientation.
senderPseudo --> class
spd --> class


In [58]:
'''3.2 constrained functional'''
# LiNGAM
model_LiNGAM = lingam.ICALiNGAM(random_state=42)
model_LiNGAM.fit(df)
print(model_LiNGAM.adjacency_matrix_)
graph_dot_model_LiNGAM = make_dot(model_LiNGAM.adjacency_matrix_, labels=node_names)
graph_dot_model_LiNGAM.format = 'png'
output_path = graph_dot_model_LiNGAM.render(filename='LiNGAM',directory='.',cleanup=True)


# Direct-LiNGAM
model_DirectLiNGAM = lingam.DirectLiNGAM(
    random_state=42,
    prior_knowledge=None,
    apply_prior_knowledge_softly=False,
    measure='pwling',
)

model_DirectLiNGAM.fit(df)
graph_dot_DirectLiNGAM = make_dot(model_DirectLiNGAM.adjacency_matrix_, labels=node_names)
graph_dot_DirectLiNGAM.format = 'png'
output_path = graph_dot_DirectLiNGAM.render(filename='DirectLiNGAM',directory='.',cleanup=True)

[[ 0.00000000e+00 -9.53452070e-02  1.29649676e+00  0.00000000e+00
   5.43605874e+01  0.00000000e+00 -2.54797364e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.56703565e+02
   6.36241908e+01  0.00000000e+00  0.00000000e+00  1.00202937e+02]
 [ 0.00000000e+00  4.42737078e+00  0.00000000e+00  0.00000000e+00
  -5.01579045e+02  0.00000000e+00  0.00000000e+00 -4.43073360e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -6.72347595e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  8.85550609e-02
   0.00000000e+00  0.00000000e+00 -7.67047767e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.02831964e-02
  -3.49949356e-02  0.00000000e+00 -1.80486825e-01  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.13490000e+00
   

In [59]:
# '''3.3 RCD'''


In [60]:
'''3.4 Boss'''
from causallearn.search.PermutationBased.BOSS import boss
from causallearn.utils.GraphUtils import GraphUtils

# G = boss(df.to_numpy(), score_func='local_score_marginal_general', node_names=node_names)
G = boss(df.to_numpy(), score_func='local_score_BIC', node_names=node_names)
pyd = GraphUtils.to_pydot(G)
pyd.write_png("BOSS.png")

BOSS edge count: 15    
BOSS completed in: 0.02s 


/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/causallearn/search/PermutationBased/BOSS.py:98: UserWarning: Using 'local_score_BIC_from_cov' instead for efficiency
  warnings.warn("Using 'local_score_BIC_from_cov' instead for efficiency")


In [61]:
'''3.5 NOTEARS'''

#notears_linear(X, lambda1, loss_type, max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3))

w = cd.notears_linear(df.values, lambda1= 0.5, loss_type='l2')
print(w)
print(type(w))
NOTEARS_adjacency_matrix_ = pd.DataFrame(w, index=node_names, columns=node_names)
# print(NOTEARS_adjacency_matrix_)
graph_dot_NOTEARS = make_dot(w, labels=node_names)
graph_dot_NOTEARS.format = 'png'
output_path = graph_dot_NOTEARS.render(filename='NOTEARS',directory='.',cleanup=True)



[[  0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           4.06220316   0.           0.
    0.           0.           0.        ]
 [  1.25783465   0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.37091017   1.27314994  -0.95710777   0.           0.
    0.           0.           0.        ]
 [  1.1208939    4.02664719  -2.4929952    0.           0.
    0.           0.           0.        ]
 [  0.           1.00727447  -0.6515864    0.           0.
    0.           0.           0.        ]
 [  0.          -0.84057319   0.55119409   0.           0.
    0.           0.           0.        ]
 [  5.80890062 105.07645417 -45.41987698   0.           0.
    0.           0.           0.        ]]
<class 'numpy.ndarray'>


In [62]:
# causal graph identify
# Obtain valid dot format
import statsmodels.api as sm

# Using the Gaussian Family for multi class
method_params = {
    "glm_family": sm.families.Gaussian()          # ≡ linear regression
}

graph_dot = utils.make_graph(w, labels=node_names)
print(graph_dot.source)
print(type(graph_dot.source))

digraph {
	sendTime
	senderPseudo
	messageID
	pos
	spd
	acl
	hed
	class
	messageID -> senderPseudo [label=4.0622031611039]
	sendTime -> messageID [label=1.257834646735903]
	sendTime -> pos [label=0.3709101731982131]
	senderPseudo -> pos [label=1.2731499434396043]
	messageID -> pos [label=-0.9571077713806321]
	sendTime -> spd [label=1.12089389710143]
	senderPseudo -> spd [label=4.0266471873575105]
	messageID -> spd [label=-2.4929952021861346]
	senderPseudo -> acl [label=1.007274465686227]
	messageID -> acl [label=-0.6515863985852673]
	senderPseudo -> hed [label=-0.8405731901096104]
	messageID -> hed [label=0.5511940929904482]
	sendTime -> class [label=5.808900621557302]
	senderPseudo -> class [label=105.07645417396465]
	messageID -> class [label=-45.41987698263677]
}

<class 'str'>


In [63]:
# Define Causal Model
model=CausalModel(
        data = df,
        treatment='sendTime',
        outcome='class',
        graph=utils.str_to_dot(graph_dot.source))

# Identification
identified_estimand = model.identify_effect(proceed_when_unidentifiable=False)
print(identified_estimand)

# Estimation
estimate = model.estimate_effect(identified_estimand,
                                # method_name="backdoor.linear_regression",
                                method_name="backdoor.generalized_linear_model",
                                # control_value=0,
                                # treatment_value=1,
                                method_params=method_params,
                                confidence_intervals=True,
                                target_units='ate',
                                test_significance=True)
print("Causal Estimate is: " + str(estimate.value))
print(estimate)



Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d               
───────────(E[class])
d[sendTime]          
Estimand assumption 1, Unconfoundedness: If U→{sendTime} and U→class then P(class|sendTime,,U) = P(class|sendTime,)

### Estimand : 2
Estimand name: iv
No such variable(s) found!

### Estimand : 3
Estimand name: frontdoor
No such variable(s) found!



/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treatin

Causal Estimate is: -3.527382616219654e-06
*** Causal Estimate ***

## Identified estimand
Estimand type: EstimandType.NONPARAMETRIC_ATE

### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d               
───────────(E[class])
d[sendTime]          
Estimand assumption 1, Unconfoundedness: If U→{sendTime} and U→class then P(class|sendTime,,U) = P(class|sendTime,)

## Realized estimand
b: class~Sigmoid(sendTime)
Target units: ate

## Estimate
Mean value: -3.527382616219654e-06
p-value: 0.032
95.0% confidence interval: (-7.438265698489488e-06, 5.199775854336508e-07)



/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treatin

In [64]:
# Refute estimand
from dowhy.causal_refuter import CausalRefuter, CausalRefutation

refuter_list = ["data_subset_refuter", 'random_common_cause', 'dummy_outcome_refuter', 'placebo_treatment_refuter','bootstrap_refuter']


ref = model.refute_estimate(
        # df,
        identified_estimand,
        estimate,
        method_name="random_common_cause",
        show_progress_bar=True,
        # optional parameters ↓
        # placebo_type="permute",          # default: permute treatment column
        # subset_fraction = 0.85,
        num_simulations = 200,
        # random_state=42,
        n_job = -1,
        # noise = 0.1
)


Refuting Estimates:   0%|          | 0/200 [00:00<?, ?it/s]/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_estimator.py:131: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  intercept_parameter = self.model.params[0]
/Users/alextang/Documents/Dev/Py_Projects/.conda_env/lib/python3.10/site-packages/dowhy/causal_estimators/regression_

In [65]:
print(ref)

Refute: Add a random common cause
Estimated effect:-3.527382616219654e-06
New effect:-3.5287652550364257e-06
p value:0.94

